# Playground

In [1]:
import os
import json
import copy

train_folder = 'data/training'
all_data = []

In [2]:
# Extracting Data from the Training Set
for filename in os.listdir(train_folder): 
    if filename.endswith('.json'):
        with open(os.path.join(train_folder, filename), 'r') as file:
            data = json.load(file)
            train_data = data['train']
            test_data = data['test']
            all_data.append([train_data, test_data])

print("Total Files:", len(os.listdir(train_folder)))
print("Total Data:", len(all_data))

Total Files: 400
Total Data: 400


In [3]:
def data_parser(user_data, hide_test_labels=False, return_answer=False):

    # Making a copy of data
    data = copy.deepcopy(user_data)
    train_samples = []
    test_sample = None
    train, test =  data

    # Parsing Samples
    for sample in train:
        train_samples.append(sample)
    for sample in test:
        answer = sample['output']
        if hide_test_labels:
            sample['output'] = '?'
            test_sample = sample
        else:
            test_sample = sample
            
    if return_answer:
        del data
        return train_samples, test_sample, answer
    else:
        del data
        return train_samples, test_sample

In [4]:
# Importing API Key
from api_keys import together_key

In [6]:
# Initializing Langchain LLama Model

from langchain_together import ChatTogether
llm = ChatTogether(api_key=together_key, model="meta-llama/Llama-3.3-70B-Instruct-Turbo")

The idea is to create a Generator Vs Discriminator model, where
- Generator: Its job is to create possible solutions(unique) which can be mapped to the output.
- Discriminator: The job of discriminator is to critique the possible solutions generated by the Generator

In [9]:
# Generator System Prompt

generator_sys_prompt = """
    You are a generator, an assistant tasked with generating possible solutions for a given pattern generation problem.
    Your goal is to produce correct solutions that align with the constraints provided in the problem.

    Problem Description:
    - You will receive an array of examples. Each example is a dictionary containing an "input" and "output" pair.
    - Your task is to identify the core pattern or principle that maps inputs to outputs.

    Constraints:
    - Do not replicate solutions already present in the provided "solutions" array.
    - Ensure your solutions are logically valid and adhere to any specific constraints mentioned in the problem.

    Reasoning Framework:
    - Treat this task like a reasoning tree:
        - The root represents the primary goal: to map inputs to outputs effectively.
        - The mid-level branches represent your reasoning process and steps to identify patterns.
        - The leaves represent potential solutions that satisfy the problem constraints.
    - Break down your reasoning step by step to ensure transparency and correctness.

    Guidelines for Solution Generation:
    - Favor symmetry and simplicity; humans tend to appreciate solutions with these qualities.
    - Verify your reasoning rigorously to ensure alignment with the given examples.
    - For every solution you provide, explain why it works.

    Output Format:
    - Provide both the solution and its reasoning in the following structured format:
        {
            "answer": [[...]],  # Your generated solution (e.g., a sequence, array, etc.)
            "reason": "Explanation of why this solution works, including your reasoning process and alignment with the problem constraints."
        }

    Additional Tips:
    - Approach the task intuitively but verify your assumptions.
    - Avoid overcomplicating the reasoning; prioritize clarity and correctness.
    - Continuously critique your reasoning and iterate to improve your solutions.
"""


In [ ]:
# Discriminator System Prompt

discriminator_sys_prompt = """
    You are a discriminator, an assistant responsible for critically evaluating solutions generated by another AI model.
    Your primary objective is to verify the correctness, logic, and alignment of the solutions with the problem constraints.
    Additionally, if possible, provide a simple, unique hint that can guide towards better solutions.

    Problem Description:
    - You will be provided with:
        - An array of examples, each consisting of an "input" and "output" pair.
        - A solution generated by another AI model.
        - A list of already tested solutions for reference.

    Evaluation Goals:
    - Ensure the generated solution:
        1. Accurately maps the inputs to the outputs as demonstrated in the given examples.
        2. Does not replicate any solution or reasoning already present in the tested solutions array.

    Testing Methodology:
    1. **Brutal Validation:**
        - Check the solution rigorously against the given examples.
        - If even one example is incorrect or the mapping is inconsistent, reject the solution outright.
        - Confirm that the solution is not a duplicate of any existing solutions in the array.
    2. **Complexity Assessment:**
        - Verify if the reasoning and solution align with human-like simplicity and intuition:
            - Humans prefer simple, easily understandable solutions that follow a top-down approach.
            - If the solution is overly complex or convoluted, it is likely unsuitable as a human-like solution.
        - Ensure the solution’s complexity does not surpass what is necessary to solve the problem effectively.
    3. **Hint Generation (if possible):**
        - If you can identify a unique and simple hint that aligns with the problem constraints and examples:
            - Ensure the hint does not completely overlap with the provided solutions, though slight overlaps are acceptable.
            - Hints should guide towards better reasoning or a possible improvement in the solution while remaining concise.
    4. **Score Assignment:**
        - Assign a score based on the quality of the solution:
            - Higher scores for solutions that are correct, simple, and intuitive.
            - Lower scores for solutions that are overly complex, even if correct.

    Output Format:
    - Provide your evaluation and hint in the following structured format:
    {
        "result": True or False,  # True if the solution is valid, False otherwise
        "score": floating value between 0 and 1,  # Reflecting the quality of the solution
        "reason": "Detailed explanation of why the solution was accepted or rejected, including evaluation of correctness, simplicity, and originality.",
        "hint": "A simple, unique hint to guide towards a better solution, if possible. If no hint is applicable, return null."
    }
"""


In [ ]:
# Aggregator System Prompt

